# scikit-multiflow

`scikit-multiflow` es un paquete de aprendizaje automático de código abierto para la transmisión de datos. Amplía las herramientas científicas disponibles en el ecosistema de Python. scikit-multiflow está pensado para aplicaciones de flujo de datos en las que los datos se generan continuamente y deben ser procesados y analizados sobre la marcha. Las muestras de datos no se almacenan, por lo que los métodos de aprendizaje están expuestos a nuevos datos sólo una vez.

## Instalar

`scikit-multiflow` está disponible a través PyPI. Así que puedes instalarlo usando el siguiente comando:

## Métodos disponibles

La siguiente imagen ilustra los múltiples métodos implementados en scikit-multiflow.

<img src="scikit-multiflow_map.png">

## Ejemplo de uso

In [1]:
%matplotlib widget

# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.drift_detection import ADWIN

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec


En este ejemplo, mostramos cómo configurar y ejecutar fácilmente experimentos en `scikit-multiflow`.

La demostración se divide en las siguientes partes:

1. Ejecución de una tarea de clasificación  
  1. Implementación de la evaluación previa
  2. La clase `EvaluatePrequential`

2. Detección de la deriva conceptual
  1. Prueba de detección de deriva
  2. Impacto en el rendimiento predictivo
  
---

## 1. Ejecutar una tarea de clasificación

En este caso utilizaremos el generador de flujos `SEA`. Un generador de datos no almacena ningún dato, sino que lo genera bajo demanda.

A continuación configuraremos un método de aprendizaje (modelo, estimador, algoritmo), en este caso el clasificador Naive Bayes:

In [11]:
stream = SEAGenerator(random_state=1)
classifier = NaiveBayes()

### Evaluación previa

La evaluación previa se implementa fácilmente como un bucle:

In [8]:
# Variables to control evaluation loop and track performance
n_samples = 0
correct_cnt = 0
max_samples = 2000

# Prequential evaluation loop
while n_samples < max_samples and stream.has_more_samples():
   X, y = stream.next_sample()      # Get one sample from the stream
   y_pred = classifier.predict(X)   # Predict class for new data
   if y[0] == y_pred[0]:
       correct_cnt += 1
   classifier.partial_fit(X, y)     # Incrementally train the model with the new data
   n_samples += 1

print('{} samples analyzed.'.format(n_samples))   
print('NaiveBayes classifier accuracy: {}'.format(correct_cnt / n_samples))

2000 samples analyzed.
NaiveBayes classifier accuracy: 0.9395


### Clase `EvaluatePrequential`

Implementa el método de evaluación prequencial y proporciona funcionalidades extra.

Hagamos el mismo experimento con los datos de la EAE pero esta vez compararemos dos clasificadores:

1. `NaiveBayes`.
2. `SGDClassifier`: SVM lineal con entrenamiento SGD.

Elegimos el `SGDClassifier` para demostrar la compatibilidad con los métodos incrementales de `scikit-learn`.

**Nota: `scikit-learn` se centra en el aprendizaje por lotes y sólo un número **limitado** de sus métodos son capaces de aprender de forma incremental.

In [9]:
# Setup stream and estimators
stream = SEAGenerator(random_state=1)
nb = NaiveBayes()
svm = SGDClassifier()

# Setup evaluator
eval = EvaluatePrequential(show_plot=True,
                           max_samples=20000,
                           metrics=['accuracy', 'kappa', 'running_time', 'model_size'])

In [10]:
# Run the evaluation
eval.evaluate(stream=stream, model=[nb, svm], model_names=['NB', 'SVM']);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 200 sample(s).
Evaluating...
 #################### [100%] [9.59s]
Processed samples: 20000
Mean performance:
NB - Accuracy     : 0.9430
NB - Kappa        : 0.8621
NB - Training time (s)  : 0.54
NB - Testing time  (s)  : 1.41
NB - Total time    (s)  : 1.96
NB - Size (kB)          : 6.8076
SVM - Accuracy     : 0.9559
SVM - Kappa        : 0.8982
SVM - Training time (s)  : 4.56
SVM - Testing time  (s)  : 1.80
SVM - Total time    (s)  : 6.36
SVM - Size (kB)          : 3.4453


---
## 2. Deriva conceptual

#### Simular un flujo de datos con deriva conceptual

Para este caso, generaremos un flujo de datos sintético concatenando 3 distribuciones de 1000 muestras cada una:
- $dist_a$: $\mu=0,8$, $\sigma=0,05$
- $dist_b$: $\mu=0,4$, $\sigma=0,02$
- $dist_c$: $\mu=0,6$, $\sigma=0,1$.

In [2]:
random_state = np.random.RandomState(12345)
dist_a = random_state.normal(0.8, 0.05, 1000)
dist_b = random_state.normal(0.4, 0.02, 1000)
dist_c = random_state.normal(0.6, 0.1, 1000)

stream = np.concatenate((dist_a, dist_b, dist_c))

# Plot the data
fig = plt.figure(figsize=(7,3), tight_layout=True)
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
ax1, ax2 = plt.subplot(gs[0]), plt.subplot(gs[1])
ax1.grid()
ax1.plot(stream, label='Stream')
ax2.grid(axis='y')
ax2.hist(dist_a, label=r'$dist_a$')
ax2.hist(dist_b, label=r'$dist_b$')
ax2.hist(dist_c, label=r'$dist_c$')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Prueba de detección de deriva

En este ejemplo utilizaremos el método de detección de deriva ADaptive WINdowing (`ADWIN`).

El objetivo es detectar que se ha producido una deriva, después de las muestras **1000** y **2000** en el flujo de datos sintéticos.

In [15]:
# Instantiate the ADWIN drift detector
drift_detector = ADWIN()

for i, val in enumerate(stream):
    drift_detector.add_element(val)        # Data is processed one sample at a time
    if drift_detector.detected_change():
        print('Change detected at index {}'.format(i))
        drift_detector.reset()

Change detected at index 1055
Change detected at index 2079


### Impacto en el rendimiento predictivo

En este caso utilizaremos dos modelos de adaptativos populares:

1. El `Hoeffding Tree` es un tipo de árbol de decisión diseñado para flujos de datos.
2. El `Hoeffding Adaptive Tree` es una mejora del `Hoeffding Tree` original.

El `Hoeffding Adaptive Tree` utiliza `ADWIN` para detectar cambios, si se detecta un cambio en una rama determinada, se crea una rama alternativa y acaba sustituyendo la rama original si muestra un mejor rendimiento con los nuevos datos.

Para este ejemplo cargaremos los datos desde un archivo csv utilizando la clase `FileStream`.

Los datos corresponden a la salida del `AGRAWALGenerator` con 3 **derivas graduales** en las marcas de 5k, 10k y 15k.

In [16]:
# Load data
stream = FileStream("agr_a_20k.csv")
# Setup estimators
cfiers = [HoeffdingTreeClassifier(), HoeffdingAdaptiveTreeClassifier()]
# Setup evaluations
eval = EvaluatePrequential(show_plot=True,
                           metrics=['accuracy', 'kappa', 'model_size'],
                           n_wait=100)

In [17]:
eval.evaluate(stream=stream, model=cfiers, model_names=['HT', 'HAT']);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 200 sample(s).
Evaluating...
 #################### [100%] [14.76s]
Processed samples: 20000
Mean performance:
HT - Accuracy     : 0.7279
HT - Kappa        : 0.4530
HT - Size (kB)          : 175.8711
HAT - Accuracy     : 0.7612
HAT - Kappa        : 0.5197
HAT - Size (kB)          : 91.9268
